In [1]:
#Parkinsonics Data Entry and Aggregation July 16 2019 - Kevin Bela Lauren

In [16]:
import pandas as pd
import numpy as np
import os 
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [17]:
df=pd.ExcelFile('raw1.xlsx')

In [72]:
# Exclude sheet 0 with all patients
listofpatients= list(df.sheet_names)[1:]
# Remove patient sheets that don't work in for loop.
# Inc and doesn't have 'Type.1' column
listofpatients.remove('063713')
listofpatients.remove('122506')

masterdf=pd.DataFrame()
counter=0
# Loop through list of patient names
for x in listofpatients:
    # tempdf is the dataframe containing 1 patient's raw data
    # tidydf is the final dataframe containing 1 patient's processed data
    counter+=1
    tidydf=pd.DataFrame()
    tempdf=df.parse(str(x))
    
    # Apply logic for each patient's dataset:
    # Drop rows with average and stdev data; not needed
    tempdf=tempdf.drop([3,4,8,9,13,14,18,19])
    tempdf=tempdf.drop('Type.1', axis=1)
    
    # Drop columns with recording filenames & patient ID
    colnames=tempdf.columns.tolist()
    del colnames[0:7]
    tempdf=tempdf[colnames]
    
    # Isolate categories one by one: A Max, A Min, E Max, E Min, Rainbow, Cookie
    # Melt to get 1 axis of all values (vertical axis)
    Amaxdf=tempdf.iloc[0:3,].melt().drop([0,1,2])
    # Specify variable names to include A_Max, and transpose to get horizontal
    maxdf1=pd.DataFrame()
    maxdf2=pd.DataFrame()
    maxdf1['variable']=['1','2','3']
    while maxdf2.shape[0]!=147:
        maxdf2=pd.concat([maxdf1, maxdf2], ignore_index=True)
    
    Amaxdf['variable']='A Max '+maxdf2['variable']+' '+Amaxdf['variable']
    Amaxdf=Amaxdf.transpose()
    tidydf=pd.concat([tidydf, Amaxdf], axis=1, ignore_index=True)
    
    # A min
    Amindf=tempdf.iloc[3:6,].melt().drop([0,1,2])
    mindf1=pd.DataFrame()
    mindf2=pd.DataFrame()
    mindf1['variable']=['1','2','3']
    while mindf2.shape[0]!=147:
        mindf2=pd.concat([mindf1, mindf2], ignore_index=True)
        
    Amindf['variable']='A Min '+mindf2['variable']+' '+Amindf['variable']
    Amindf=Amindf.transpose()
    tidydf=pd.concat([tidydf, Amindf], axis=1, ignore_index=True)
    
    #E max
    Emaxdf=tempdf.iloc[6:9,].melt().drop([0,1,2])
    maxdf3=pd.DataFrame()
    maxdf4=pd.DataFrame()
    maxdf3['variable']=['1','2','3']
    while maxdf4.shape[0]!=147:
        maxdf4=pd.concat([maxdf3, maxdf4], ignore_index=True)
    
    Emaxdf['variable']='E Max '+maxdf4['variable']+' '+Emaxdf['variable']
    Emaxdf=Emaxdf.transpose()
    tidydf=pd.concat([tidydf, Emaxdf], axis=1, ignore_index=True)
    
    #E min
    Emindf=tempdf.iloc[9:12,].melt().drop([0,1,2])
    mindf3=pd.DataFrame()
    mindf4=pd.DataFrame()
    mindf3['variable']=['1','2','3']
    while mindf4.shape[0]!=147:
        mindf4=pd.concat([mindf3, mindf4], ignore_index=True)
    Emindf['variable']='E Min '+mindf4['variable']+' '+Emindf['variable']
    Emindf=Emindf.transpose()
    tidydf=pd.concat([tidydf, Emindf], axis=1, ignore_index=True)
    
    # Rainbow
    # raindf=tempdf.iloc[12,].to_frame()
    # raindf=raindf.transpose()
    # raindf.columns='Rainbow '+raindf.columns
    # raindf.drop(columns=['Rainbow Type'])
    # tidydf=pd.concat([tidydf, raindf], axis=1, ignore_index=True)
    
    # Append tidydf to masterdf, which will have processed data for all patients - 1 per row
    if x=='013916':
        masterdf=masterdf.append(tidydf)
    else:
        masterdf=masterdf.append(tidydf.iloc[1,])
    
        

In [74]:
# Concatenate list of patient IDS with masterdf so that data is on row with IDs
listofpatients.insert(0, 0)
masterdf['ID']=listofpatients

In [75]:
# Move the ID column to the front
cols=list(masterdf)
cols.insert(0, cols.pop(cols.index('ID')))
masterdf=masterdf.loc[:,cols]

In [77]:
masterdf.to_excel(r'C:\Users\kevin\Documents\JHU\Labs\Parkinsonics\Parkinsonics_FINAL_MasterVM.xlsx')